In [1]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import numpy as np
import pandas as pd

from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn import preprocessing

from statsmodels.tsa.stattools import coint

from scipy import stats

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

In [3]:
Tickers = list(df[0].Symbol)

In [4]:
import quandl
quandl.ApiConfig.api_key = 'ZtrVMzQmPvx8zec1J4r9'

# get the table for daily stock prices and,
# filter the table for selected tickers, columns within a time range
# set paginate to True because Quandl limits tables API to 10,000 rows per call

data = quandl.get_table('WIKI/PRICES', ticker = Tickers[0:50], 
                        qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, 
                        date = { 'gte': '2015-12-31', 'lte': '2016-12-31' }, 
                        paginate=True)
data.head()

,ticker,date,adj_close
None,,,
0,MO,2016-12-30,65.839853
1,MO,2016-12-29,66.063798
2,MO,2016-12-28,65.732749
3,MO,2016-12-27,66.034588
4,MO,2016-12-23,65.937220


In [5]:
data = data.pivot(index='date', columns='ticker', values='adj_close')

In [6]:
data.head()

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,APH,ARE,ATVI,AWK,AXP,GOOG,GOOGL,LNT,MMM,MO
date,,,,,,,,,,,,,,,,,,,,,
2015-12-31,41.083301,41.610225,150.050177,101.696810,54.965678,100.204026,90.28,42.818272,100.419635,93.94,...,51.396200,85.664638,38.169244,57.376282,67.156318,758.88,778.01,29.272393,144.053187,54.656153
2016-01-04,39.982768,40.195379,151.774892,101.783763,53.453287,98.426229,85.24,40.930493,97.853890,91.97,...,50.057911,84.479592,37.094471,57.923637,65.263775,741.84,759.44,29.192709,140.400218,53.886216
2016-01-05,39.845201,39.812192,150.738069,99.233131,53.230603,99.865858,85.00,40.920958,98.363195,92.34,...,49.497010,85.645677,36.621176,57.760391,64.259569,742.58,761.53,29.323954,141.012234,54.975396
2016-01-06,40.022072,40.509790,146.750290,97.291172,53.239882,98.339271,85.30,40.577726,98.171004,91.02,...,48.326008,85.067375,36.276065,57.510720,62.482895,743.62,759.33,29.361453,138.172099,55.557543
2016-01-07,38.322141,39.743415,148.375310,93.185040,53.082148,95.199139,81.92,39.605233,95.288144,89.11,...,46.584264,82.251704,35.763330,57.078598,61.920336,726.39,741.00,29.052089,134.806013,54.590427


In [7]:
returns = data.pct_change()

In [8]:
returns

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,APH,ARE,ATVI,AWK,AXP,GOOG,GOOGL,LNT,MMM,MO
date,,,,,,,,,,,,,,,,,,,,,
2015-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-04,-0.026788,-0.034002,0.011494,0.000855,-0.027515,-0.017742,-0.055826,-0.044088,-0.025550,-0.020971,...,-0.026039,-0.013834,-0.028158,0.009540,-0.028181,-0.022454,-0.023869,-0.002722,-0.025358,-0.014087
2016-01-05,-0.003441,-0.009533,-0.006831,-0.025059,-0.004166,0.014626,-0.002816,-0.000233,0.005205,0.004023,...,-0.011205,0.013803,-0.012759,-0.002818,-0.015387,0.000998,0.002752,0.004496,0.004359,0.020213
2016-01-06,0.004439,0.017522,-0.026455,-0.019570,0.000174,-0.015286,0.003529,-0.008388,-0.001954,-0.014295,...,-0.023658,-0.006752,-0.009424,-0.004323,-0.027648,0.001401,-0.002889,0.001279,-0.020141,0.010589
2016-01-07,-0.042475,-0.018918,0.011073,-0.042205,-0.002963,-0.031932,-0.039625,-0.023966,-0.029366,-0.020984,...,-0.036042,-0.033099,-0.014134,-0.007514,-0.009003,-0.023170,-0.024140,-0.010536,-0.024362,-0.017407
2016-01-08,-0.010513,-0.001978,-0.021971,0.005288,-0.027268,-0.005785,0.032471,-0.020944,-0.009681,-0.014140,...,-0.014998,-0.013716,-0.015440,0.003869,-0.003289,-0.016410,-0.013617,-0.000323,-0.003405,0.001720
2016-01-11,-0.016844,0.017587,0.010236,0.016192,-0.031806,-0.039812,0.004966,0.001475,0.010489,0.017416,...,-0.007720,-0.012504,0.014002,0.009720,0.006601,0.002183,0.002955,0.003228,-0.000214,0.020948
2016-01-12,0.006589,0.022395,0.006936,0.014513,0.017817,0.001276,0.039059,0.017923,0.017938,0.004923,...,0.025070,-0.011361,0.025131,-0.000996,0.005464,0.014022,0.016738,-0.008044,0.002848,-0.001345
2016-01-13,-0.034826,-0.045238,-0.039914,-0.025710,-0.056346,-0.011043,-0.028419,-0.022190,-0.017325,-0.016923,...,-0.018554,-0.027053,-0.060884,-0.001661,-0.024068,-0.035134,-0.034575,0.000649,-0.015192,-0.026945


In [9]:
returns = returns.iloc[1:,:].dropna(axis=1)

In [10]:
returns

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,APH,ARE,ATVI,AWK,AXP,GOOG,GOOGL,LNT,MMM,MO
date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,-0.026788,-0.034002,0.011494,0.000855,-0.027515,-0.017742,-0.055826,-0.044088,-0.025550,-0.020971,...,-0.026039,-0.013834,-0.028158,0.009540,-0.028181,-0.022454,-0.023869,-0.002722,-0.025358,-0.014087
2016-01-05,-0.003441,-0.009533,-0.006831,-0.025059,-0.004166,0.014626,-0.002816,-0.000233,0.005205,0.004023,...,-0.011205,0.013803,-0.012759,-0.002818,-0.015387,0.000998,0.002752,0.004496,0.004359,0.020213
2016-01-06,0.004439,0.017522,-0.026455,-0.019570,0.000174,-0.015286,0.003529,-0.008388,-0.001954,-0.014295,...,-0.023658,-0.006752,-0.009424,-0.004323,-0.027648,0.001401,-0.002889,0.001279,-0.020141,0.010589
2016-01-07,-0.042475,-0.018918,0.011073,-0.042205,-0.002963,-0.031932,-0.039625,-0.023966,-0.029366,-0.020984,...,-0.036042,-0.033099,-0.014134,-0.007514,-0.009003,-0.023170,-0.024140,-0.010536,-0.024362,-0.017407
2016-01-08,-0.010513,-0.001978,-0.021971,0.005288,-0.027268,-0.005785,0.032471,-0.020944,-0.009681,-0.014140,...,-0.014998,-0.013716,-0.015440,0.003869,-0.003289,-0.016410,-0.013617,-0.000323,-0.003405,0.001720
2016-01-11,-0.016844,0.017587,0.010236,0.016192,-0.031806,-0.039812,0.004966,0.001475,0.010489,0.017416,...,-0.007720,-0.012504,0.014002,0.009720,0.006601,0.002183,0.002955,0.003228,-0.000214,0.020948
2016-01-12,0.006589,0.022395,0.006936,0.014513,0.017817,0.001276,0.039059,0.017923,0.017938,0.004923,...,0.025070,-0.011361,0.025131,-0.000996,0.005464,0.014022,0.016738,-0.008044,0.002848,-0.001345
2016-01-13,-0.034826,-0.045238,-0.039914,-0.025710,-0.056346,-0.011043,-0.028419,-0.022190,-0.017325,-0.016923,...,-0.018554,-0.027053,-0.060884,-0.001661,-0.024068,-0.035134,-0.034575,0.000649,-0.015192,-0.026945
2016-01-14,0.020347,0.011222,-0.005909,0.021871,0.066041,0.006227,0.024123,0.020357,0.026395,0.016988,...,0.003437,-0.003322,0.015491,0.013147,0.007001,0.020212,0.016426,0.015073,0.017734,0.007615


In [11]:
N_PRIN_COMPONENTS = 2
pca = PCA(n_components=N_PRIN_COMPONENTS)
pca.fit(returns)

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [12]:
pca.components_.T.shape

(50, 2)

In [13]:
X = preprocessing.StandardScaler().fit_transform(pca.components_.T)

In [14]:
print(X.shape)

(50, 2)


In [15]:
kmeans = KMeans(n_clusters=4)  
kmeans.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=4, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [16]:
clustered = kmeans.labels_

In [17]:
def find_cointegrated_pairs(data, significance=0.05):
    # This function is from https://www.quantopian.com/lectures/introduction-to-pairs-trading
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            result = coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < significance:
                pairs.append((keys[i], keys[j]))
    return score_matrix, pvalue_matrix, pairs

In [18]:
ticker_count = len(returns.columns)
print ("Total pairs possible in universe: %d " % (ticker_count*(ticker_count-1)/2))

Total pairs possible in universe: 1225 


In [19]:
clustered_series = pd.Series(index=returns.columns, data=clustered.flatten())
clustered_series_all = pd.Series(index=returns.columns, data=clustered.flatten())
clustered_series = clustered_series[clustered_series != -1]

In [21]:
CLUSTER_SIZE_LIMIT = 9999
counts = clustered_series.value_counts()
ticker_count_reduced = counts[(counts>1) & (counts<=CLUSTER_SIZE_LIMIT)]
print("Clusters formed: %d" % len(ticker_count_reduced))
print("Pairs to evaluate: %d" % (ticker_count_reduced*(ticker_count_reduced-1)).sum())

Clusters formed: 3
Pairs to evaluate: 952


In [22]:
counts

3    26
1    17
0     6
2     1
dtype: int64

In [23]:
cluster_dict = {}
for i, which_clust in enumerate(ticker_count_reduced.index):
    tickers = clustered_series[clustered_series == which_clust].index
    score_matrix, pvalue_matrix, pairs = find_cointegrated_pairs(
        data[tickers]
    )
    cluster_dict[which_clust] = {}
    cluster_dict[which_clust]['score_matrix'] = score_matrix
    cluster_dict[which_clust]['pvalue_matrix'] = pvalue_matrix
    cluster_dict[which_clust]['pairs'] = pairs

In [24]:
pairs = []
for clust in cluster_dict.keys():
    pairs.extend(cluster_dict[clust]['pairs'])

In [25]:
print("We found %d pairs." % len(pairs))

We found 64 pairs.


In [26]:
pairs

[('A', 'ABBV'),
 ('A', 'ALB'),
 ('A', 'AMZN'),
 ('ABBV', 'AES'),
 ('ABBV', 'ALB'),
 ('ABBV', 'ALLE'),
 ('ABBV', 'ANSS'),
 ('ABBV', 'ATVI'),
 ('ABMD', 'AMZN'),
 ('ACN', 'AGN'),
 ('ADBE', 'AMZN'),
 ('ADS', 'AES'),
 ('ADS', 'AGN'),
 ('ADS', 'AIG'),
 ('ADS', 'ALB'),
 ('ADS', 'ALGN'),
 ('ADS', 'ALK'),
 ('ADS', 'ALLE'),
 ('ADS', 'AME'),
 ('ADS', 'AMGN'),
 ('ADS', 'AMZN'),
 ('ADS', 'ANSS'),
 ('ADS', 'ANTM'),
 ('ADS', 'AOS'),
 ('ADS', 'APD'),
 ('ADS', 'APH'),
 ('ADS', 'ATVI'),
 ('AES', 'ALLE'),
 ('AES', 'AOS'),
 ('AGN', 'AIG'),
 ('ALGN', 'ANSS'),
 ('ALGN', 'AOS'),
 ('ALXN', 'AMGN'),
 ('ALXN', 'AMZN'),
 ('ALXN', 'ANSS'),
 ('ALXN', 'ANTM'),
 ('ALXN', 'AOS'),
 ('ALXN', 'APD'),
 ('ALXN', 'APH'),
 ('ALXN', 'ATVI'),
 ('AME', 'AMGN'),
 ('AME', 'AMZN'),
 ('AME', 'ANSS'),
 ('AME', 'ANTM'),
 ('AME', 'AOS'),
 ('AME', 'APD'),
 ('AME', 'APH'),
 ('AME', 'ATVI'),
 ('ANSS', 'AOS'),
 ('ANSS', 'APD'),
 ('ANSS', 'APH'),
 ('ABC', 'AXP'),
 ('AEE', 'ALL'),
 ('AEE', 'MMM'),
 ('AEE', 'MO'),
 ('AFL', 'AIV'),
 ('AFL', 